# 文档工具

c语言使用[doxygen](http://www.doxygen.nl/)来从代码注释中提取文档,同时其支持markdown格式

doxygen支持多种格式,下面给出一个相对比价好上手的格式作为标准:

## 文件注释，放于文件的开头


文件注释主要是使用`@xxx`标识的变量

```C
/** 
* @file         filename 
* @brief        This is a brief description. 
* @details  This is the detail description. 
* @author       author 
* @date     date 
* @version  A001 
* @par Copyright (c):  
*       XXX公司 
* @par History:          
*   version: author, date, desc\n 
*/  
```

## 函数注释，放于函数声明前

函数注释主要是使用`@xxx`标识的变量

```C
/** 
* This is a brief description. 
* This is a detail description. 
* @param[in]   inArgName input argument description. 
* @param[out]  outArgName output argument description.  
* @retval  OK  成功 
* @retval  ERROR   错误  
* @par 标识符 
*      保留 
* @par 其它 
*      无 
* @par 修改日志 
*      XXX于201X-XX-XX创建 
*/
```

## 数据结构注释，放于数据结构定义前

```c
/** 
 * The brief description. 
 * The detail description. 
 */  
typedef struct  
{  
    int var1;///<Description of the member variable  
}XXXX;
```

## 宏定义注释，放于宏定义上方或者右侧

```c
/** Description of the macro */  
#define XXXX_XXX_XX     0
```
或者

```c
#define XXXX_XXX_XX     0 ///< Description of the macro.  
```

## 全局和静态变量注释

```c
/**  Description of global variable  */  
int g_xxx = 0;  

static int s_xxx = 0; ///<  Description of static variable
``` 

一般来说文档工具是为自动生成接口文档设计的,因此一般来说这个工具只要对头文件使用即可.

### 例子:为`BINARY_VECTOR`写注释

In [1]:
%%writefile ../codes/C1_C_tools/doc_tools/include/binary_vector.h
/** 
* @file         binary_vector.h
* @brief        binary_vector接口文件
* @author       author 
* @date     2018-1-8
* @version  dev
* @par Copyright (c):  
*       XXX公司 
* @par History:          
*   version: author, date, desc\n 
*/
#ifndef BINARY_VECTOR_H
#define BINARY_VECTOR_H//一般是文件名的大写 头文件结尾写上一行

/** 
* @struct       BINARY_VECTOR
* @brief        vector结构
*/  
typedef struct {
    float x; ///头一位
    float y; ///尾一位
} BINARY_VECTOR;

/** 
* @fn float VEC_mod(BINARY_VECTOR)
* @brief 求模 
* @param[vector]   要求模的向量  
* @return  float 模的值
* @par 修改日志 
*      XXX于201X-XX-XX创建 
*/
extern float VEC_mod(BINARY_VECTOR);

/** 
* @fn BINARY_VECTOR VEC_add(BINARY_VECTOR,BINARY_VECTOR)
* @brief 求两个向量的和
* @param[vector]   求和的第一个向量
* @param[vector]   求和的第二个向量  
* @return  BINARY_VECTOR 相加后的向量
* @par 修改日志 
*      XXX于201X-XX-XX创建 
*/
extern BINARY_VECTOR VEC_add(BINARY_VECTOR,BINARY_VECTOR);

/** 
* @fn float VEC_mul(BINARY_VECTOR ,BINARY_VECTOR)
* @brief 求积
* @param[vector]   求积的第一个向量
* @param[vector]   求积的第二个向量  
* @return  float 内积的值
* @par 修改日志 
*      XXX于201X-XX-XX创建 
*/
extern float VEC_mul(BINARY_VECTOR ,BINARY_VECTOR);

#endif

Overwriting ../codes/C1_C_tools/doc_tools/include/binary_vector.h


## 使用doxygen生成文档

在有了注释以后,还需要使用doxygen工具生成所需的文档形式.使用命令行工具`doxygen`.其步骤为:

1. `doxygen -g`生成一份配置模板
2. 修改配置模板中的参数,中要的有:

    + 项目名称 `PROJECT_NAME = "MyProject"`
    + 对C项目优化 `OPTIMIZE_OUTPUT_FOR_C = YES`
    + 源文件路径 `INPUT = /home/tom/project/include`
    + 源文件编码 `INPUT_ENCODEING = UTF-8`
    + 生成XML `GENERATE_XML = YES` 如果要使用breathe扩展，所以必须生成xml
    + XML输出目录 `XML_OUTPUT = my_xml`
    + 是否需要生成html `GENERATE_HTML = NO`
    + 是否需要生成latex `GENERATE_LATEX = NO`
    + 是否显示包含文件 `SHOW_INCLUDE_FILES = NO`
    
    一般来说如果要单独使用doxygen则需要设置 `GENERATE_HTML = Yes`,如果要配合`breathe`则需要设置`GENERATE_XML = YES`
    
3. 运行`doxygen [Doxyfile]`生成目标文档


## 使用breathe将doxygen与sphinx结合

作为python用户,文档工具的首选肯定是sphinx,要让这两个结合需要使用[breathe](https://breathe.readthedocs.io/en/latest/index.html)


其后续步骤为:

+ 配置`conf.py`,以支持breathe扩展.前提是这一扩展已经安装.

    加入扩展:
```python
extensions = ['breathe']
```

+ 在`conf.py`中配置`breathe`:

```python
breathe_projects = { "myproject": "./my_xml" }
breathe_default_project = "myproject"
breathe_domain_by_extension = {"h" : "c"}
```

+ 在需要的`.rst`文件中加入以下代码引入接口

按breathe扩展语法编写`rst`文件,比如:

```rst
My API
==========

sp.h
----

.. doxygenfile:: sp.h

```

+ 运行 make html 生成html即可.

在这些步骤中，需要先调用doxygen生成xml，再调用sphinx-build(make html时自动调用)生成最终文档，如果想一步完成，可以在Makefile中加入doxygen调用:

```make
html:
    doxygen Doxyfile
    $(SPHINXBUILD) -b html $(ALLSPHINXOPTS) $(BUILDDIR)/html
    @echo
    @echo "Build finished. The HTML pages are in $(BUILDDIR)/html."
```